# Lectura de archivos

In [1]:
import pandas as pd
import os
import glob

In [2]:
# --- CONFIGURACIÓN ---
carpeta_fuente = "Fuente de Ventas"
hoja_especifica = "INDICADOR"
lista_dataframes = [] 


patron = os.path.join(carpeta_fuente, "*.xlsx")
archivos = glob.glob(patron)

print(f"📂 Se encontraron {len(archivos)} archivos en '{carpeta_fuente}'")

for ruta_archivo in archivos:
    nombre_archivo = os.path.basename(ruta_archivo)
    print(f"\n 1. Procesando: {nombre_archivo}")
    
    try:
        
        df_temp = pd.read_excel(
            ruta_archivo, 
            sheet_name=hoja_especifica, 
            header=0
        )
        
        # Quitamos las filas vacías
        df_temp.columns = df_temp.columns.astype(str)
        df_temp = df_temp.loc[:, ~df_temp.columns.str.contains('^Unnamed')]
        
        
        df_temp['archivo_origen'] = nombre_archivo
        
        
        cols_monetarias = [c for c in df_temp.columns if 'MOVIL' in str(c).upper() or 'TOTAL' in str(c).upper()]
        
        for col in cols_monetarias:
            if df_temp[col].dtype == 'object':
                df_temp[col] = (
                    df_temp[col]
                    .astype(str)
                    .str.replace('S/', '', regex=False)
                    .str.replace(',', '', regex=False)
                    .str.replace(' ', '', regex=False)
                )
                df_temp[col] = pd.to_numeric(df_temp[col], errors='coerce')

        
        df_temp = df_temp.dropna(subset=['MOVIL'], how='all') if 'MOVIL' in df_temp.columns else df_temp

        
        lista_dataframes.append(df_temp)
        print(f" 2. {nombre_archivo} procesado con {len(df_temp)} filas.")

    except Exception as e:
        print(f"Error en {nombre_archivo}: {e}")



📂 Se encontraron 61 archivos en 'Fuente de Ventas'

 1. Procesando: BD CENTRAL - LIQ#01 29.12 AL 04.01.xlsx
 2. BD CENTRAL - LIQ#01 29.12 AL 04.01.xlsx procesado con 12163 filas.

 1. Procesando: BD CENTRAL - LIQ#01.xlsx
 2. BD CENTRAL - LIQ#01.xlsx procesado con 11926 filas.

 1. Procesando: BD CENTRAL - LIQ#02 05.01 AL 11.01.xlsx
 2. BD CENTRAL - LIQ#02 05.01 AL 11.01.xlsx procesado con 15239 filas.

 1. Procesando: BD CENTRAL - LIQ#02.xlsx
 2. BD CENTRAL - LIQ#02.xlsx procesado con 14534 filas.

 1. Procesando: BD CENTRAL - LIQ#03 12.01 AL 18.01.xlsx
 2. BD CENTRAL - LIQ#03 12.01 AL 18.01.xlsx procesado con 16880 filas.

 1. Procesando: BD CENTRAL - LIQ#03.xlsx
 2. BD CENTRAL - LIQ#03.xlsx procesado con 14636 filas.

 1. Procesando: BD CENTRAL - LIQ#04 19.01 AL 25.01.xlsx
 2. BD CENTRAL - LIQ#04 19.01 AL 25.01.xlsx procesado con 16301 filas.

 1. Procesando: BD CENTRAL - LIQ#04.xlsx
 2. BD CENTRAL - LIQ#04.xlsx procesado con 15226 filas.

 1. Procesando: BD CENTRAL - LIQ#05 26.01 AL

In [3]:
len(lista_dataframes)

61

In [4]:
columnas_finales = [
    "IdServicio", "IdConductor", "Flota", "COMISION DIRECTO", "VEHICULO SOLICITADO", 
    "MEMBRESÍA", "IdMovil", "PlacaAuto", "ModeloAuto", "Conductor", "Fecha_Servicio", 
    "Hora_Servicio", "DIA", "TIPO", "Revisado", "Total_Vale", "PEAJE + PARQUEO", 
    "MONTO TARIFA", "MONTO A LIQUIDAR", "COMISION CONDUCTOR", "COSTO DE VENTA", 
    "Observacion", "NombreComercial", "NombreCompleto", "DescModoReserva", 
    "DescTipoPago", "DescTipoServicio", "DirOrigen", "ZonaOrigen", "DistritoOrigen", 
    "Promocion", "Descuento Promocion", "TipoUsuario", "Estrellas", "Zonas", 
    "TIPO_DE_SERVICIO:", "ITINERARIO", "UNIDAD DE NEGOCIO", "GRUPOS", 
    "TIPO DE SERVICIO", "Total Propina", "BonoConductor", "Total_Otro"
]

In [5]:
# Uniendo Dataframes en uno solo
lista_limpia = []

for i, df in enumerate(lista_dataframes):
    
    cols_presentes = [c for c in columnas_finales if c in df.columns]
    
    df_filtrado = df[cols_presentes].loc[:, ~df[cols_presentes].columns.duplicated()].copy()
    
    lista_limpia.append(df_filtrado)
    
    #staus de carga
    if i % 10 == 0:
        print(f"Procesados {i} de {len(lista_dataframes)}...")


df_final = pd.concat(lista_limpia, ignore_index=True)

print(f" ¡Listo! DataFrame final consolidado con {len(df_final)} filas.")

Procesados 0 de 61...
Procesados 10 de 61...
Procesados 20 de 61...
Procesados 30 de 61...
Procesados 40 de 61...
Procesados 50 de 61...
Procesados 60 de 61...
 ¡Listo! DataFrame final consolidado con 1003044 filas.


In [6]:
len(df_final)

1003044

In [7]:
df_final.head()

,IdServicio,IdConductor,Flota,COMISION DIRECTO,VEHICULO SOLICITADO,MEMBRESÍA,PlacaAuto,ModeloAuto,Conductor,Fecha_Servicio,...,TIPO_DE_SERVICIO:,ITINERARIO,UNIDAD DE NEGOCIO,GRUPOS,TIPO DE SERVICIO,Total Propina,BonoConductor,Total_Otro,IdMovil,DIA
0,5436243,2978.0,FLOTA PROPIA,0.279,VAN,NO,CSH676,REFINE,Jorge Luis Montenegro Vergara,2025-12-29,...,NaN,NaN,URBANO,CORPORATIVO,CORPORATIVO,0.0,NaN,23.9,NaN,NaN
1,5438628,2978.0,FLOTA PROPIA,0.279,AUTO,NO,CSH676,REFINE,Jorge Luis Montenegro Vergara,2025-12-29,...,NaN,NaN,DESTINO,CORPORATIVO,CORPORATIVO,0.0,NaN,5.9,NaN,NaN
2,5437787,2978.0,FLOTA PROPIA,0.279,AUTO,NO,CSH676,REFINE,Jorge Luis Montenegro Vergara,2025-12-29,...,NaN,NaN,DESTINO,PARTICULAR,PARTICULAR,0.0,NaN,27.0,NaN,NaN
3,5422269,2978.0,FLOTA PROPIA,0.279,AUTO,NO,CSH676,REFINE,Jorge Luis Montenegro Vergara,2025-12-31,...,NaN,NaN,DESTINO,CORPORATIVO,CORPORATIVO,0.0,NaN,32.5,NaN,NaN
4,5431114,2978.0,FLOTA PROPIA,0.279,AUTO,NO,CSH676,REFINE,Jorge Luis Montenegro Vergara,2025-12-31,...,NaN,NaN,DESTINO,PARTICULAR,PARTICULAR,0.0,NaN,33.0,NaN,NaN


In [8]:
df_final['IdMovil']=df_final['IdMovil'].astype(str)
df_final['Flota']=df_final['Flota'].astype(str)
df_final['IdServicio']=df_final['IdServicio'].astype(str)
df_final['IdConductor']=df_final['IdConductor'].astype(str)
df_final.dtypes

IdServicio                     object
IdConductor                    object
Flota                          object
COMISION DIRECTO              float64
VEHICULO SOLICITADO            object
MEMBRESÍA                      object
PlacaAuto                      object
ModeloAuto                     object
Conductor                      object
Fecha_Servicio         datetime64[ns]
Hora_Servicio                  object
TIPO                           object
Revisado                       object
Total_Vale                    float64
PEAJE + PARQUEO               float64
MONTO TARIFA                   object
MONTO A LIQUIDAR              float64
COMISION CONDUCTOR            float64
COSTO DE VENTA                float64
Observacion                    object
NombreComercial                object
NombreCompleto                 object
DescModoReserva                object
DescTipoPago                   object
DescTipoServicio               object
DirOrigen                      object
ZonaOrigen  

In [9]:
df_final[df_final['IdServicio'] == '5437300']

,IdServicio,IdConductor,Flota,COMISION DIRECTO,VEHICULO SOLICITADO,MEMBRESÍA,PlacaAuto,ModeloAuto,Conductor,Fecha_Servicio,...,TIPO_DE_SERVICIO:,ITINERARIO,UNIDAD DE NEGOCIO,GRUPOS,TIPO DE SERVICIO,Total Propina,BonoConductor,Total_Otro,IdMovil,DIA
930820,5437300,3621.0,AEROLINEAS,0.25,AUTO,NO,CJI139,Versa,ULISES ANTONIO GONZALES HUARCAYA,2025-12-28,...,NaN,NaN,DESTINO,PARTICULAR,PARTICULAR,0.0,NaN,11.0,nan,NaN


# Subida de archivos al servidor

In [10]:
import pandas as pd
import pypyodbc as odbc
from sqlalchemy import create_engine, types
from sqlalchemy.engine import URL, create_engine

# Parámetros de conexión (guardarlo)
servername = 'LP-MP2P84ND-PE'
database = 'Marketing'
username = 'sa'
password = '@nalistaDirecto'
# El driver (verificar si es el más actual)
driver = 'ODBC Driver 17 for SQL Server' 

# Creación de la URL de conexión para SQLAlchemy
connection_string = f"DRIVER={{{driver}}};SERVER={servername};DATABASE={database};UID={username};PWD={password}"
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

engine = create_engine(connection_url, fast_executemany=True)


In [11]:
df_final = df_final[df_final['IdServicio'] != 'nan']

In [12]:
df_final[df_final['IdServicio'] == 'nan']

,IdServicio,IdConductor,Flota,COMISION DIRECTO,VEHICULO SOLICITADO,MEMBRESÍA,PlacaAuto,ModeloAuto,Conductor,Fecha_Servicio,...,TIPO_DE_SERVICIO:,ITINERARIO,UNIDAD DE NEGOCIO,GRUPOS,TIPO DE SERVICIO,Total Propina,BonoConductor,Total_Otro,IdMovil,DIA


In [13]:
#Tablas a cargar
tb_data = {
    'tb_servicios': df_final
}

In [14]:
from sqlalchemy import create_engine
import urllib


def cargar_multiples_tablas_seguro(tb_data, engine):
    for nombre_tabla, df in tb_data.items():
        try:
            print(f"🚀 Iniciando carga optimizada de {nombre_tabla}...")
            
            # Configuración de conexión e interación con BD
            with engine.connect() as conn:
                # Considerar CHUNKSIZE para carga masiva de lo contrario demora
                df.to_sql(
                    nombre_tabla, 
                    con=conn, 
                    if_exists='replace', 
                    index=False, 
                    chunksize=20000 # Bloques de 20k registros
                )
                conn.commit() # Aseguramos que se guarde
                
            print(f" ----> {len(df)} filas subidas correctamente.")

        except Exception as e:
            print(f"Error crítico: {e}")
            #considera liberar por si falla 
            engine.dispose()

cargar_multiples_tablas_seguro(tb_data, engine)

🚀 Iniciando carga optimizada de tb_servicios...
 ----> 986129 filas subidas correctamente.


In [15]:
engine.dispose() 